In [0]:
 from pyspark.sql.functions import *

In [0]:
d1=[["1","11-01-2020","2020-07-01","2020-07-01 12:01:19.111"],["2","21-05-2019","2019-01-11","2019-06-24 12:01:19.222"],["3","19-11-2021","2021-12-07","2021-11-16 16:44:55.406"]]
df=spark.createDataFrame(d1,["id","sample_date","sample_date1","sample_timestamp"])
df.show(truncate=False)
df.printSchema()
  

+---+-----------+------------+-----------------------+
|id |sample_date|sample_date1|sample_timestamp       |
+---+-----------+------------+-----------------------+
|1  |11-01-2020 |2020-07-01  |2020-07-01 12:01:19.111|
|2  |21-05-2019 |2019-01-11  |2019-06-24 12:01:19.222|
|3  |19-11-2021 |2021-12-07  |2021-11-16 16:44:55.406|
+---+-----------+------------+-----------------------+

root
 |-- id: string (nullable = true)
 |-- sample_date: string (nullable = true)
 |-- sample_date1: string (nullable = true)
 |-- sample_timestamp: string (nullable = true)



In [0]:
'''
Default date format expected is yyyy-MM-dd  i.e 2020-01-11
If the date or timestamp is not in that format, need to specify the format explicitly.
'''

df=df.withColumn("sample_date",to_date("sample_date",format = 'dd-MM-yyyy')) \
      .withColumn("sample_date1",to_date("sample_date1")) \
      .withColumn("sample_timestamp",to_timestamp("sample_timestamp"))

df.show(truncate=False)
df.printSchema()

+---+-----------+------------+-----------------------+
|id |sample_date|sample_date1|sample_timestamp       |
+---+-----------+------------+-----------------------+
|1  |2020-01-11 |2020-07-01  |2020-07-01 12:01:19.111|
|2  |2019-05-21 |2019-01-11  |2019-06-24 12:01:19.222|
|3  |2021-11-19 |2021-12-07  |2021-11-16 16:44:55.406|
+---+-----------+------------+-----------------------+

root
 |-- id: string (nullable = true)
 |-- sample_date: date (nullable = true)
 |-- sample_date1: date (nullable = true)
 |-- sample_timestamp: timestamp (nullable = true)



In [0]:
'''
Creati temporary view to check sql queries
'''

df.createOrReplaceTempView("date_testing")
spark.sql("select * from date_testing;").show(truncate=False)

+---+-----------+------------+-----------------------+
|id |sample_date|sample_date1|sample_timestamp       |
+---+-----------+------------+-----------------------+
|1  |2020-01-11 |2020-07-01  |2020-07-01 12:01:19.111|
|2  |2019-05-21 |2019-01-11  |2019-06-24 12:01:19.222|
|3  |2021-11-19 |2021-12-07  |2021-11-16 16:44:55.406|
+---+-----------+------------+-----------------------+



# Date transformations

In [0]:
'''
Get the date in any format required using date_format
''' 
print("Using Pyhton \n","*"*20)
df2=df.select("sample_date").withColumn("formatted_date",date_format("sample_date","MM/dd/yyyy"))
df2.show()
df2.printSchema()

print("Using SQL \n","*"*20)
df2=spark.sql("select sample_date,date_format(sample_date,'MM/dd/yyyy') as formatted_date from date_testing;")
df2.show()
df2.printSchema()

Using Pyhton 
 ********************
+-----------+--------------+
|sample_date|formatted_date|
+-----------+--------------+
| 2020-01-11|    01/11/2020|
| 2019-05-21|    05/21/2019|
| 2021-11-19|    11/19/2021|
+-----------+--------------+

root
 |-- sample_date: date (nullable = true)
 |-- formatted_date: string (nullable = true)

Using SQL 
 ********************
+-----------+--------------+
|sample_date|formatted_date|
+-----------+--------------+
| 2020-01-11|    01/11/2020|
| 2019-05-21|    05/21/2019|
| 2021-11-19|    11/19/2021|
+-----------+--------------+

root
 |-- sample_date: date (nullable = true)
 |-- formatted_date: string (nullable = true)



In [0]:
'''
Timestamp to date
''' 
print("Using Pyhton \n","*"*20)
df2=df.select("sample_timestamp").withColumn("date",to_date("sample_timestamp")) \
                                 .withColumn("formatted_date",date_format("sample_timestamp",format="dd-MM-yyyy"))
df2.show(truncate=False)
df2.printSchema()

print("Using SQL \n","*"*20)
df2=spark.sql("""select sample_timestamp,
                        to_date(sample_timestamp) as date, 
                        date_format(sample_timestamp,'dd-MM-yyyy') as formatted_date 
                 from date_testing;""")
df2.show(truncate=False)
df2.printSchema()

Using Pyhton 
 ********************
+-----------------------+----------+--------------+
|sample_timestamp       |date      |formatted_date|
+-----------------------+----------+--------------+
|2020-07-01 12:01:19.111|2020-07-01|01-07-2020    |
|2019-06-24 12:01:19.222|2019-06-24|24-06-2019    |
|2021-11-16 16:44:55.406|2021-11-16|16-11-2021    |
+-----------------------+----------+--------------+

root
 |-- sample_timestamp: timestamp (nullable = true)
 |-- date: date (nullable = true)
 |-- formatted_date: string (nullable = true)

Using SQL 
 ********************
+-----------------------+----------+--------------+
|sample_timestamp       |date      |formatted_date|
+-----------------------+----------+--------------+
|2020-07-01 12:01:19.111|2020-07-01|01-07-2020    |
|2019-06-24 12:01:19.222|2019-06-24|24-06-2019    |
|2021-11-16 16:44:55.406|2021-11-16|16-11-2021    |
+-----------------------+----------+--------------+

root
 |-- sample_timestamp: timestamp (nullable = true)
 |-- dat

In [0]:
'''
Add interval to dates
''' 
print("Using Pyhton \n","*"*20)
df2=df.select('sample_date',
              expr('sample_date + INTERVAL 10 DAYS').alias("add_date1"),
              date_add("sample_date",10).alias("add_date2"),
              expr('sample_date + INTERVAL 2 MONTHS as add_months'),
             )
df2.show(truncate=False)
df2.printSchema()

print("Using SQL \n","*"*20)
df2=spark.sql("""select sample_date,
                        sample_date + INTERVAL 10 DAYS as add_date1,
                        date_add(sample_date,10) as add_date2,
                        sample_date + INTERVAL 10 MONTHS as add_months
                 from date_testing;""")
df2.show(truncate=False)
df2.printSchema()



Using Pyhton 
 ********************
+-----------+----------+----------+----------+
|sample_date|add_date1 |add_date2 |add_months|
+-----------+----------+----------+----------+
|2020-01-11 |2020-01-21|2020-01-21|2020-03-11|
|2019-05-21 |2019-05-31|2019-05-31|2019-07-21|
|2021-11-19 |2021-11-29|2021-11-29|2022-01-19|
+-----------+----------+----------+----------+

root
 |-- sample_date: date (nullable = true)
 |-- add_date1: date (nullable = true)
 |-- add_date2: date (nullable = true)
 |-- add_months: date (nullable = true)

Using SQL 
 ********************
+-----------+----------+----------+----------+
|sample_date|add_date1 |add_date2 |add_months|
+-----------+----------+----------+----------+
|2020-01-11 |2020-01-21|2020-01-21|2020-11-11|
|2019-05-21 |2019-05-31|2019-05-31|2020-03-21|
|2021-11-19 |2021-11-29|2021-11-29|2022-09-19|
+-----------+----------+----------+----------+

root
 |-- sample_date: date (nullable = true)
 |-- add_date1: date (nullable = true)
 |-- add_date2: date 

In [0]:
'''
Fetch YEAR, MONTH, DAY...
SUNDAY - 0 , SATURDAY - 7
''' 
print("Using Pyhton \n","*"*20)
df2=df.select('sample_date',
              year("sample_date").alias("year"),
              month("sample_date").alias("month"),
              dayofmonth("sample_date").alias("day"),              
              dayofweek("sample_date").alias("day of week"),
              weekofyear("sample_date").alias("weak of year"),
              quarter("sample_date").alias("quarter"),
              next_day("sample_date","Friday").alias("Date of upcoming Friday"),
             )
df2.show(truncate=False)
df2.printSchema()

print("Using SQL \n","*"*20)
df2=spark.sql("""select sample_date,
                        year(sample_date) as date_year,
                        month(sample_date) as month,
                        dayofmonth(sample_date) as day,
                        dayofweek(sample_date) as day_of_week,
                        weekofyear(sample_date) as week_of_year,
                        quarter(sample_date) as quarter,
                        next_day(sample_date,'Friday') as date_of_upcoming_Friday
                 from date_testing;""")
df2.show(truncate=False)
df2.printSchema()



Using Pyhton 
 ********************
+-----------+----+-----+---+-----------+------------+-------+-----------------------+
|sample_date|year|month|day|day of week|weak of year|quarter|Date of upcoming Friday|
+-----------+----+-----+---+-----------+------------+-------+-----------------------+
|2020-01-11 |2020|1    |11 |7          |2           |1      |2020-01-17             |
|2019-05-21 |2019|5    |21 |3          |21          |2      |2019-05-24             |
|2021-11-19 |2021|11   |19 |6          |46          |4      |2021-11-26             |
+-----------+----+-----+---+-----------+------------+-------+-----------------------+

root
 |-- sample_date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- day of week: integer (nullable = true)
 |-- weak of year: integer (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- Date of upcoming Friday: date (nullable = true)

Using SQL 
 *********

In [0]:
'''
Dates diff and add
''' 
print("Using Pyhton \n","*"*20)
df2=df.select('sample_date',
              datediff(current_date(),"sample_date").alias("date_diff_in_days"),
              months_between(current_date(),"sample_date").alias("months_between"),
              date_add("sample_date",5).alias("date+5"),
              date_add("sample_date",-5).alias("date-5"),
              date_sub("sample_date",-5).alias("date+5_1"),
              date_sub("sample_date",5).alias("date-5_1"),
              add_months("sample_date",20).alias("add_months"),
              add_months("sample_date",-40).alias("subtract_months"), 
              
              # set to the first of that month or year
              trunc("sample_date","Month").alias("truncate_month"),
              trunc("sample_date","Year").alias("truncate_year")
             )
df2.show(truncate=False)
df2.printSchema()

print("Using SQL \n","*"*20)
df2=spark.sql("""select sample_date,
                        datediff(current_date(),sample_date) as date_diff_in_days,
                        months_between(current_date(),sample_date) as months_between,
                        date_add(sample_date,5) as date_add_5,
                        date_add(sample_date,-5) as date_sub_5,
                        date_sub(sample_date,-5) as date_add_5_1,
                        date_sub(sample_date,5) as date_sub_5_1,
                        add_months(sample_date,20) as add_months,
                        add_months(sample_date,-40) as subtract_months, 
                        --set to the first of that month or year
                        trunc(sample_date,'Month') as truncate_month,
                        trunc(sample_date,'Year') as truncate_year
                 from date_testing;""")
df2.show(truncate=False)
df2.printSchema()



Using Pyhton 
 ********************
+-----------+-----------------+--------------+----------+----------+----------+----------+----------+---------------+--------------+-------------+
|sample_date|date_diff_in_days|months_between|date+5    |date-5    |date+5_1  |date-5_1  |add_months|subtract_months|truncate_month|truncate_year|
+-----------+-----------------+--------------+----------+----------+----------+----------+----------+---------------+--------------+-------------+
|2020-01-11 |1059             |34.80645161   |2020-01-16|2020-01-06|2020-01-16|2020-01-06|2021-09-11|2016-09-11     |2020-01-01    |2020-01-01   |
|2019-05-21 |1294             |42.48387097   |2019-05-26|2019-05-16|2019-05-26|2019-05-16|2021-01-21|2016-01-21     |2019-05-01    |2019-01-01   |
|2021-11-19 |381              |12.5483871    |2021-11-24|2021-11-14|2021-11-24|2021-11-14|2023-07-19|2018-07-19     |2021-11-01    |2021-01-01   |
+-----------+-----------------+--------------+----------+----------+----------+---

# Timestamp  transformations

In [0]:
'''
Add interval to Timestamps
''' 
print("Using Pyhton \n","*"*20)
df2=df.select('sample_timestamp',
              to_date('sample_timestamp').alias('timestamp_to_date'),
              expr('sample_timestamp + INTERVAL 10 SECONDS').alias("add_timestamp1"),
              date_add("sample_timestamp",10).alias("add_timestamp2"),
              expr('sample_timestamp + INTERVAL 2 MONTHS as add_months'),
              # converting to unix_timestamp and subtracting
              ((current_timestamp().cast("long")-col('sample_timestamp').cast("long"))/60).alias("timestamp_diff_in_days")
             )
df2.show(truncate=False)
df2.printSchema()

print("Using SQL \n","*"*20)
df2=spark.sql("""select sample_timestamp,
                        to_date(sample_timestamp) as timestamp_to_date,
                        sample_timestamp + INTERVAL 10 SECONDS as add_timestamp1,
                        date_add(sample_timestamp,10) as add_timestamp2,
                        sample_timestamp + INTERVAL 10 MONTHS as add_months,
                        (cast(current_timestamp() as int)-cast(sample_timestamp as int))/60 as timestamp_diff_in_days
                 from date_testing;""")
df2.show(truncate=False)
df2.printSchema()



Using Pyhton 
 ********************
+-----------------------+-----------------+-----------------------+--------------+-----------------------+----------------------+
|sample_timestamp       |timestamp_to_date|add_timestamp1         |add_timestamp2|add_months             |timestamp_diff_in_days|
+-----------------------+-----------------+-----------------------+--------------+-----------------------+----------------------+
|2020-07-01 12:01:19.111|2020-07-01       |2020-07-01 12:01:29.111|2020-07-11    |2020-09-01 12:01:19.111|1277445.4833333334    |
|2019-06-24 12:01:19.222|2019-06-24       |2019-06-24 12:01:29.222|2019-07-04    |2019-08-24 12:01:19.222|1814565.4833333334    |
|2021-11-16 16:44:55.406|2021-11-16       |2021-11-16 16:45:05.406|2021-11-26    |2022-01-16 16:44:55.406|552841.8833333333     |
+-----------------------+-----------------+-----------------------+--------------+-----------------------+----------------------+

root
 |-- sample_timestamp: timestamp (nullable = tru

In [0]:
'''
Fetch YEAR, MONTH, DAY...
SUNDAY - 0 , SATURDAY - 7
''' 
print("Using Pyhton \n","*"*20)
df.select("sample_timestamp",
          current_timestamp(),
          hour("sample_timestamp").alias("hours"),
          minute("sample_timestamp").alias("minute"),
          second("sample_timestamp").alias("seconds"),
          month("sample_timestamp").alias("month"),
          year("sample_timestamp").alias("year"),
          dayofweek("sample_timestamp").alias("dat of week"),
          dayofmonth("sample_timestamp").alias("date of month"),
          weekofyear("sample_timestamp").alias("weak of year"),
          quarter("sample_timestamp").alias("quarter")        
          ).show(truncate=0)


print("Using Pyhton \n","*"*20)

spark.sql(""" select sample_timestamp,
                current_timestamp(),
                hour(sample_timestamp) as hours,
                minute(sample_timestamp) as minute,
                second(sample_timestamp) as seconds,
                month(sample_timestamp) as month,
                year(sample_timestamp) as year1,
                dayofweek(sample_timestamp) as dat_of_week,
                dayofmonth(sample_timestamp) as date_of_month,
                weekofyear(sample_timestamp) as weak_of_year,
                quarter(sample_timestamp) as quarter
               from date_testing;
""").show(truncate=0)

Using Pyhton 
 ********************
+-----------------------+-----------------------+-----+------+-------+-----+----+-----------+-------------+------------+-------+
|sample_timestamp       |current_timestamp()    |hours|minute|seconds|month|year|dat of week|date of month|weak of year|quarter|
+-----------------------+-----------------------+-----+------+-------+-----+----+-----------+-------------+------------+-------+
|2020-07-01 12:01:19.111|2022-12-05 14:11:23.159|12   |1     |19     |7    |2020|4          |1            |27          |3      |
|2019-06-24 12:01:19.222|2022-12-05 14:11:23.159|12   |1     |19     |6    |2019|2          |24           |26          |2      |
|2021-11-16 16:44:55.406|2022-12-05 14:11:23.159|16   |44    |55     |11   |2021|3          |16           |46          |4      |
+-----------------------+-----------------------+-----+------+-------+-----+----+-----------+-------------+------------+-------+

Using Pyhton 
 ********************
+-----------------------

## Timestamp to Unix timestamp

In [0]:
print("Using Pyhton \n","*"*20)
df2=df.select('sample_timestamp',
              unix_timestamp('sample_timestamp').alias("unix1"),              
              unix_timestamp('sample_timestamp',format="yyyy-MM-dd HH:mm:ss").alias("unix2"),
              col('sample_timestamp').cast("long").alias("unix3")
             )
df2.show(truncate=False)
df2.printSchema()

print("Using SQL \n","*"*20)
df2=spark.sql("""select sample_timestamp,
                        unix_timestamp(sample_timestamp) as unix1,
                        unix_timestamp(sample_timestamp,'yyyy-MM-dd HH:mm:ss') as unix2,
                        to_unix_timestamp(sample_timestamp, 'yyyy-MM-dd HH:mm:ss') as unix3,
                        to_unix_timestamp(sample_timestamp) as unix4,
                        unix_timestamp(TIMESTAMP '2020-02-01 11:01:19.06') as unix5,                        
                        cast(sample_timestamp as int) as unix6
                 from date_testing;""")
df2.show(truncate=False)
df2.printSchema()



Using Pyhton 
 ********************
+-----------------------+----------+----------+----------+
|sample_timestamp       |unix1     |unix2     |unix3     |
+-----------------------+----------+----------+----------+
|2020-07-01 12:01:19.111|1593604879|1593604879|1593604879|
|2019-06-24 12:01:19.222|1561377679|1561377679|1561377679|
|2021-11-16 16:44:55.406|1637081095|1637081095|1637081095|
+-----------------------+----------+----------+----------+

root
 |-- sample_timestamp: timestamp (nullable = true)
 |-- unix1: long (nullable = true)
 |-- unix2: long (nullable = true)
 |-- unix3: long (nullable = true)

Using SQL 
 ********************
+-----------------------+----------+----------+----------+----------+----------+----------+
|sample_timestamp       |unix1     |unix2     |unix3     |unix4     |unix5     |unix6     |
+-----------------------+----------+----------+----------+----------+----------+----------+
|2020-07-01 12:01:19.111|1593604879|1593604879|1593604879|1593604879|1580554879

# Date to Unix timestamp

In [0]:
print("Using Pyhton \n","*"*20)
df2=df.select('sample_date',
              unix_timestamp('sample_date').alias("unix1"),              
              unix_timestamp('sample_date',format="yyyy-MM-dd HH:mm:ss").alias("unix2"),
              col('sample_date').cast("long").alias("unix3")
             )
df2.show(truncate=False)
df2.printSchema()

print("Using SQL \n","*"*20)
df2=spark.sql("""select sample_date,
                        unix_timestamp(sample_date) as unix1,
                        unix_timestamp(sample_date,'yyyy-MM-dd HH:mm:ss') as unix2,
                        to_unix_timestamp(sample_date, 'yyyy-MM-dd HH:mm:ss') as unix3,
                        to_unix_timestamp(sample_date) as unix4,
                        unix_timestamp(TIMESTAMP '2020-02-01 11:01:19.06') as unix5,                        
                        cast(sample_date as int) as unix6
                 from date_testing;""")
df2.show(truncate=False)
df2.printSchema()




Using Pyhton 
 ********************
+-----------+----------+----------+-----+
|sample_date|unix1     |unix2     |unix3|
+-----------+----------+----------+-----+
|2020-01-11 |1578700800|1578700800|null |
|2019-05-21 |1558396800|1558396800|null |
|2021-11-19 |1637280000|1637280000|null |
+-----------+----------+----------+-----+

root
 |-- sample_date: date (nullable = true)
 |-- unix1: long (nullable = true)
 |-- unix2: long (nullable = true)
 |-- unix3: long (nullable = true)

Using SQL 
 ********************
+-----------+----------+----------+----------+----------+----------+-----+
|sample_date|unix1     |unix2     |unix3     |unix4     |unix5     |unix6|
+-----------+----------+----------+----------+----------+----------+-----+
|2020-01-11 |1578700800|1578700800|1578700800|1578700800|1580554879|null |
|2019-05-21 |1558396800|1558396800|1558396800|1558396800|1580554879|null |
|2021-11-19 |1637280000|1637280000|1637280000|1637280000|1580554879|null |
+-----------+----------+----------

# Unix timestamp to timestamp

In [0]:
print("Using Pyhton \n","*"*20)
df2=df.withColumn("unix_timestamp",unix_timestamp('sample_timestamp')) \
      .withColumn("unix_date",unix_timestamp('sample_date')) \
      .select('sample_timestamp', 'unix_timestamp',             
              from_unixtime("unix_timestamp").alias("back_to_timestamp"),
              'sample_date', 'unix_date',
              from_unixtime("unix_date").alias("back_to_date")
             )
df2.show(truncate=False)
df2.printSchema()

print("Using SQL \n","*"*20)
df2=spark.sql("""select sample_timestamp,unix_timestamp_val,
                        from_unixtime(unix_timestamp_val) as back_to_timestamp,
                        sample_date,unix_date,
                        from_unixtime(unix_date) as back_to_date
                  from 
                      (
                        select sample_timestamp,sample_date,
                        unix_timestamp(sample_timestamp) as unix_timestamp_val ,
                         unix_timestamp(sample_date) as unix_date
                        from date_testing
                      )
                        """)
df2.show(truncate=False)
df2.printSchema()



Using Pyhton 
 ********************
+-----------------------+--------------+-------------------+-----------+----------+-------------------+
|sample_timestamp       |unix_timestamp|back_to_timestamp  |sample_date|unix_date |back_to_date       |
+-----------------------+--------------+-------------------+-----------+----------+-------------------+
|2020-07-01 12:01:19.111|1593604879    |2020-07-01 12:01:19|2020-01-11 |1578700800|2020-01-11 00:00:00|
|2019-06-24 12:01:19.222|1561377679    |2019-06-24 12:01:19|2019-05-21 |1558396800|2019-05-21 00:00:00|
|2021-11-16 16:44:55.406|1637081095    |2021-11-16 16:44:55|2021-11-19 |1637280000|2021-11-19 00:00:00|
+-----------------------+--------------+-------------------+-----------+----------+-------------------+

root
 |-- sample_timestamp: timestamp (nullable = true)
 |-- unix_timestamp: long (nullable = true)
 |-- back_to_timestamp: string (nullable = true)
 |-- sample_date: date (nullable = true)
 |-- unix_date: long (nullable = true)
 |-- b

In [0]:
df.select(from_unixtime(lit(-2209095000000/1000)),unix_timestamp(to_date(lit("1780-02-02")))).show()

+-----------------------------------------------+--------------------------------------------------------+
|from_unixtime(-2.209095E9, yyyy-MM-dd HH:mm:ss)|unix_timestamp(to_date(1780-02-02), yyyy-MM-dd HH:mm:ss)|
+-----------------------------------------------+--------------------------------------------------------+
|                            1899-12-30 18:30:00|                                             -5993049600|
|                            1899-12-30 18:30:00|                                             -5993049600|
|                            1899-12-30 18:30:00|                                             -5993049600|
+-----------------------------------------------+--------------------------------------------------------+

